Leemos los datos del csv

In [1]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
import pandas as pd
df = pd.read_csv('data/desaparecidos.csv',header=0,names=['estado','municipio','cvegeo','year','total','tasa'], encoding='utf-8')

Agrupamos y quitamos los índices de las columnas estado y year para poder usarlas en el pivot:

In [3]:
df.groupby(['estado','year']).sum().reset_index('year').reset_index('estado').head()

,estado,year,cvegeo,total,tasa
0,Aguascalientes,2006,11066,0,0.000000
1,Aguascalientes,2007,11066,25,24.059914
2,Aguascalientes,2008,11066,7,3.226498
3,Aguascalientes,2009,11066,20,3.495144
4,Aguascalientes,2010,11066,18,7.733581


Creamos la pivot table para tener los años en columnas:

In [4]:
tmp = df.groupby(['estado','year']).sum().reset_index('year').reset_index('estado')
estado_tiempo = tmp.pivot(index='estado',columns='year',values='total')
estado_tiempo.tail()

year,2006,2007,2008,2009,2010,2011,2012,2013,2014
estado,,,,,,,,,
Tamaulipas,15,76,66,207,901,1227,1071,761,831
Tlaxcala,0,0,0,2,0,1,2,6,3
Veracruz de Ignacio de la Llave,2,3,2,1,22,59,45,115,211
Yucatán,0,1,1,2,4,6,10,30,29
Zacatecas,0,1,6,24,30,34,16,46,32


En la maniobra perdimos las claves cvegeo que luego vamos a necesitar para unirlo a las geometrías. Recuperémoslas:

In [5]:
cve_estado = df[['estado','cvegeo']].groupby('estado').first()
cve_estado.tail()

,cvegeo
estado,
Tamaulipas,28001
Tlaxcala,29001
Veracruz de Ignacio de la Llave,30001
Yucatán,31001
Zacatecas,32001


Además tenemos que reindexar las tablas para poder usar los valores de las columnas 'estado' como llaves en la unión:

In [6]:
cve_estado = cve_estado.reset_index('estado')
estado_tiempo = estado_tiempo.reset_index('estado')

Ahora si unimos:

In [7]:
estado_tiempo = pd.merge(cve_estado,estado_tiempo, on='estado')
estado_tiempo.head()

,estado,cvegeo,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aguascalientes,1001,0,25,7,20,18,30,14,15,62
1,Baja California,2001,0,7,25,11,8,19,128,177,381
2,Baja California Sur,3001,0,0,2,1,3,2,3,8,3
3,Campeche,4001,0,0,5,0,1,0,0,10,59
4,Chiapas,7001,0,0,0,5,6,9,12,31,41


Ahora leemos el shapefile con las geometrías de los estados:

In [8]:
from geopandas import GeoDataFrame
gf = GeoDataFrame.from_file('data/mexico_entidad.shp', encoding='utf-8')
gf.columns

Index([u'CVEGEO', u'NOMBRE', u'OID', u'POB1', u'POB10', u'POB10_R', u'POB11', u'POB11_R', u'POB12', u'POB12_R', u'POB13', u'POB13_R', u'POB14', u'POB14_R', u'POB15', u'POB15_R', u'POB16', u'POB16_R', u'POB17', u'POB17_R', u'POB18', u'POB18_R', u'POB19', u'POB19_R', u'POB2', u'POB20', u'POB20_R', u'POB21', u'POB21_R', u'POB22', u'POB22_R', u'POB23', u'POB23_R', u'POB24', u'POB24_R', u'POB25', u'POB25_R', u'POB26_R', u'POB27_R', u'POB28_R', u'POB29_R', u'POB2_R', u'POB3', u'POB30_R', u'POB31', u'POB31_R', u'POB32', u'POB32_R', u'POB33', u'POB33_R', u'POB34', u'POB34_R', u'POB35', u'POB35_R', u'POB36', u'POB36_R', u'POB37', u'POB37_R', u'POB38', u'POB38_R', u'POB39', u'POB39_R', u'POB3_R', u'POB4', u'POB40', u'POB40_R', u'POB41', u'POB41_R', u'POB42', u'POB42_R', u'POB43', u'POB43_R', u'POB44', u'POB44_R', u'POB45', u'POB45_R', u'POB46', u'POB46_R', u'POB47', u'POB47_R', u'POB48', u'POB48_R', u'POB49', u'POB49_R', u'POB4_R', u'POB5', u'POB50', u'POB50_R', u'POB51', u'POB51_R', u'POB52', u

Sólo queremos las columnas CVEGEO, POB1 y geometry:

In [10]:
gf[['POB1','CVEGEO','geometry']].tail()

,POB1,CVEGEO,geometry
27,2748391,05,"POLYGON ((2469954.193300001 1978522.9934, 2469..."
28,650555,06,"(POLYGON ((1158658.7249 767185.3242000006, 115..."
29,4796580,07,"POLYGON ((3560747.32 695090.7327000014, 356114..."
30,3406465,08,"POLYGON ((1961651.183200002 2201097.236400001,..."
31,8851080,09,"POLYGON ((2802175.6602 843326.7184000015, 2801..."


Trístemente las cvegeo no coinciden, tendremos que intentar unir por nombre. 
Nos quedamos con las columnas necesarias del shape y renombramos la columna nombre para usarla como llave común:

In [11]:
gf = gf[['NOMBRE','POB1','geometry']]
gf = gf.rename(columns = {'NOMBRE':'estado'})
gf.columns

Index([u'estado', u'POB1', u'geometry'], dtype='object')

Unimos los dos conjuntos de datos y lo casteamos a GeoDataFrame:

In [12]:
des_estado = pd.merge(gf,estado_tiempo, on='estado')
des_estado = GeoDataFrame(des_estado)
des_estado.head()

,estado,POB1,geometry,cvegeo,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aguascalientes,1184996,"POLYGON ((2470517.824100003 1155028.587099999,...",1001,0,25,7,20,18,30,14,15,62
1,Durango,1632934,"POLYGON ((2107050.444499999 1641943.398400001,...",10001,2,13,19,39,151,61,45,21,19
2,Guanajuato,5486372,"POLYGON ((2566856.637900002 1091418.135699999,...",11001,5,89,56,88,41,129,108,190,371
3,Guerrero,3388768,(POLYGON ((2723198.210000001 539350.5295999981...,12001,3,16,15,8,18,38,48,114,248
4,Hidalgo,2665018,"POLYGON ((2862894.734499998 1047165.223700002,...",13001,3,10,17,28,27,97,34,16,2


Necesitamos darle valor a la proyección

In [15]:
gf.crs

{'init': u'epsg:4326'}

Antes de guardar el resultado como shape necesitamos renombrar las columnas de los años porque son enteros y no strings.



In [13]:
des_estado = des_estado.rename(columns={2006:'2006',2007:'2007',2008:'2008',2009:'2009',2010:'2010',2011:'2011',2012:'2012',2013:'2013',2014:'2014'})


Guardamos el resultado como shapefile para leerlo con clusterpy

In [14]:
des_estado.to_file('data/desaparecidos_estatal',encoding='utf-8')

In [18]:
for c in des_estado.columns:
    print des_estado[c].dtype
    